# Working with the Polar Data   

In [1]:
import scipy.signal as sig
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib
import os
import pandas as pd 
import numpy as np
from datetime import datetime
import xarray as xr

Using matplotlib backend: Qt5Agg


## Grabbing the Polar h5 and converting it into a Panda's data frame. 
I will be working with 64 rotation  

In [ ]:
fn_polar_data = r"radar_rot_5.h5"
#h5 = pd.HDFStore(fn_polar_data)
#h5.keys()
polar_pd_dataframe =  pd.read_hdf(fn_polar_data,key='/polar_image', start=0, stop=262144)

### Using numpy to split the polar data into 64 dataframes

In [ ]:
list_rot_df = np.split(polar_pd_dataframe, 64)

In [22]:
#rot_dict = {}
#for index, x in enumerate(list_rot_df):
#    rot_dict[index] = x

### Xarray
Using xArray to concat the 64 data frame into one and 
selecting a subset distance and the angle (256*256) of the polar data 

In [ ]:
def create_a_xr_dataarray(rotation_list, DateTimeIndex_range=list(range(4096)), Distance_Cells_Index_range = list(range(1024))):
    list_x_dataarray = []
    for index, x in enumerate(rotation_list[0:]):
        list_x_dataarray.append(xr.DataArray(x.reset_index(drop=True),coords=[DateTimeIndex_range, Distance_Cells_Index_range], dims=['DateTime', 'Distance_Cells']))
    
    xr_dataframe_rot = xr.concat(list_x_dataarray, dim='rotation_id') 
    
    return select_a_subset_of_the_dataarray(xr_dataframe_rot)

def select_a_subset_of_the_dataarray(rotation_dataarray, rotation_id_range = list(range(0,64)), DateTimeRange = list(range(3269,3525)), Distance_Cells_Range = list(range(215,471))):
    boy_locations_polar_data = rotation_dataarray[rotation_id_range, DateTimeRange, Distance_Cells_Range]
    return boy_locations_polar_data
    


In [7]:
boy_locations_polar_data = create_a_xr_dataarray(list_rot_df)

### Working with the welch algorithm

In [10]:
def create_frequency_welch_df(backscatters_for_point_over_time, frequency_of_samples=1.3):
    return create_welch_df(backscatters_for_point_over_time, frequency_of_samples)

def create_wavelength_welch_df(backscatters_returns_from_one_pulse, distance_between_cells=5.995):
    return create_welch_df(backscatters_returns_from_one_pulse, distance_between_cells)

def create_welch_df(dataframe, spacing):
    """Takes a single column spacing is either in metres or seconds"""
    x,y = sig.welch(dataframe, spacing, noverlap=None)
    dic = {'x':x,'y':y}
    welch_df = pd.DataFrame(dic)
    return welch_df

def calc_welch(welch_df, psd_dic = {}, save = False, line_name = ''):
    """ Given a dataframe will run welch's methods on the values, printing out the maxmimum power spectral density and the 
        frequency it occurs at, plots the output and returns the data
    """
    psd_dic[line_name] = [welch_df.x.max(), welch_df.y.loc[welch_df.y==welch_df.y.max()]]
    plt.plot(welch_df.x,welch_df.y, label=line_name)
    plt.ylabel('S(f) m^2/Hz')
    plt.xlabel('Frequency (Hz)')
    plt.legend()
    return psd_dic

In [14]:
# Converting the second rotation into a data frame 
df = boy_locations_polar_data[60].to_pandas()

# Using the Welch algorithm to convert the wave form time domain to the frequency domain 
for x in range(len(df)):
    welch_df = create_wavelength_welch_df(df.iloc[x])
    wavelength_df = calc_welch(welch_df)

C:\Users\le12jm\AppData\Local\Continuum\Anaconda3\lib\site-packages\matplotlib\axes\_axes.py:545: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [ ]:
create_welch_df(df, 1.3)